In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import initializers
import keras.backend as K
import numpy as np
import pandas as pd
from tensorflow.keras.layers import *
from keras.regularizers import l2#正则化

Using TensorFlow backend.


In [2]:
# 12-0.2
# 13-2.4
# 18-12.14
import pandas as pd
import numpy as np
normal = np.loadtxt(r'E:\水泵代码调试\试验数据(包括压力脉动和振动)\2013.9.12-未发生缠绕前\2013-9.12振动\2013-9-12振动-1250rmin-mat\1250rnormalvib4.txt', delimiter=',')
chanrao = np.loadtxt(r'E:\水泵代码调试\试验数据(包括压力脉动和振动)\2013.9.17-发生缠绕后\振动\9-18上午振动1250rmin-mat\1250r_chanraovib4.txt', delimiter=',')
print(normal.shape,chanrao.shape,"***************************************************")
data_normal=normal[6:8]   #提取前两行
data_chanrao=chanrao[6:8]   #提取前两行
print(data_normal.shape,data_chanrao.shape)
print(data_normal,"\r\n",data_chanrao,"***************************************************")
data_normal=data_normal.reshape(1,-1)
data_chanrao=data_chanrao.reshape(1,-1)
print(data_normal.shape,data_chanrao.shape)
print(data_normal,"\r\n",data_chanrao,"***************************************************")

(22, 32768) (20, 32768) ***************************************************
(2, 32768) (2, 32768)
[[ 0.35374 -0.4776  -0.29668 ... -0.20514 -0.4609  -0.15571]
 [ 0.78149 -0.33487 -0.70813 ... -0.11674  0.30481  0.13308]] 
 [[ 1.393    -0.72383  -0.67901  ... -1.1614    0.33641  -0.75729 ]
 [ 0.035352 -0.96852  -0.32823  ... -1.1896   -0.47507  -0.77645 ]] ***************************************************
(1, 65536) (1, 65536)
[[ 0.35374 -0.4776  -0.29668 ... -0.11674  0.30481  0.13308]] 
 [[ 1.393   -0.72383 -0.67901 ... -1.1896  -0.47507 -0.77645]] ***************************************************


In [3]:
#水泵的两种故障类型信号normal正常，chanrao故障
data_normal=data_normal.reshape(-1, 512)#(65536,1)-(128, 515)
data_chanrao=data_chanrao.reshape(-1,512)
print(data_normal.shape,data_chanrao.shape)


(128, 512) (128, 512)


In [4]:
import numpy as np
def yuchuli(data,label):#(4:1)(51:13)
    #打乱数据顺序
    np.random.shuffle(data)
    train = data[0:102,:]
    test = data[102:128,:]
    label_train = np.array([label for i in range(0,102)])
    label_test =np.array([label for i in range(0,26)])
    return train,test ,label_train ,label_test
def stackkk(a,b,c,d,e,f,g,h):
    aa = np.vstack((a, e))
    bb = np.vstack((b, f))
    cc = np.hstack((c, g))
    dd = np.hstack((d, h))
    return aa,bb,cc,dd
x_tra0,x_tes0,y_tra0,y_tes0 = yuchuli(data_normal,0)
x_tra1,x_tes1,y_tra1,y_tes1 = yuchuli(data_chanrao,1)
tr1,te1,yr1,ye1=stackkk(x_tra0,x_tes0,y_tra0,y_tes0 ,x_tra1,x_tes1,y_tra1,y_tes1)

x_train=tr1
x_test=te1
y_train = yr1
y_test = ye1

#打乱数据
state = np.random.get_state()
np.random.shuffle(x_train)
np.random.set_state(state)
np.random.shuffle(y_train)

state = np.random.get_state()
np.random.shuffle(x_test)
np.random.set_state(state)
np.random.shuffle(y_test)


#对训练集和测试集标准化
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x
x_train=ZscoreNormalization(x_train)
x_test=ZscoreNormalization(x_test)
# print(x_test[0])


#转化为一维序列
x_train = x_train.reshape(-1,512,1)
x_test = x_test.reshape(-1,512,1)
print(x_train.shape,x_test.shape)

def to_one_hot(labels,dimension=2):
    results = np.zeros((len(labels),dimension))
    for i,label in enumerate(labels):
        results[i,label] = 1
    return results
one_hot_train_labels = to_one_hot(y_train)
one_hot_test_labels = to_one_hot(y_test)


(204, 512, 1) (52, 512, 1)


In [5]:
#定义挤压函数
def squash(vectors, axis=-1):
    """
    对向量的非线性激活函数
    ## vectors: some vectors to be squashed, N-dim tensor
    ## axis: the axis to squash
    :return: a Tensor with same shape as input vectors
    """
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors

class Length(layers.Layer):
    """
    计算向量的长度。它用于计算与margin_loss中的y_true具有相同形状的张量
    Compute the length of vectors. This is used to compute a Tensor that has the same shape with y_true in margin_loss
    inputs: shape=[dim_1, ..., dim_{n-1}, dim_n]
    output: shape=[dim_1, ..., dim_{n-1}]
    """
    def call(self, inputs, **kwargs):
        return K.sqrt(K.sum(K.square(inputs), -1))

    def compute_output_shape(self, input_shape):
         return input_shape[:-1]
        
    def get_config(self):
        config = super(Length, self).get_config()
        return config
#定义预胶囊层
def PrimaryCap(inputs, dim_capsule, n_channels, kernel_size, strides, padding):
    """
    进行普通二维卷积 `n_channels` 次, 然后将所有的胶囊重叠起来
    :param inputs: 4D tensor, shape=[None, width, height, channels]
    :param dim_capsule: the dim of the output vector of capsule
    :param n_channels: the number of types of capsules
    :return: output tensor, shape=[None, num_capsule, dim_capsule]
    """
    output = layers.Conv2D(filters=dim_capsule*n_channels, kernel_size=kernel_size, strides=strides,
                           padding=padding,name='primarycap_conv2d')(inputs)
    outputs = layers.Reshape(target_shape=[-1, dim_capsule], name='primarycap_reshape')(output)
    return layers.Lambda(squash, name='primarycap_squash')(outputs)

class DenseCapsule(layers.Layer):
    """
    胶囊层. 输入输出都为向量. 
    ## num_capsule: 本层包含的胶囊数量
    ## dim_capsule: 输出的每一个胶囊向量的维度
    ## routings: routing 算法的迭代次数
    """
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_initializer='glorot_uniform',**kwargs):
        super(DenseCapsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_initializer = kernel_initializer

    def build(self, input_shape):
        assert len(input_shape) >= 3, '输入的 Tensor 的形状[None, input_num_capsule, input_dim_capsule]'#(None,1152,8)
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]

        #转换矩阵
        self.W = self.add_weight(shape=[self.num_capsule, self.input_num_capsule,
                                        self.dim_capsule, self.input_dim_capsule],
                                initializer=self.kernel_initializer,name='W')
        self.built = True

    def call(self, inputs, training=None):
        # inputs.shape=[None, input_num_capsuie, input_dim_capsule]
        # inputs_expand.shape=[None, 1, input_num_capsule, input_dim_capsule]
        inputs_expand = K.expand_dims(inputs, 1)
        # 运算优化:将inputs_expand重复num_capsule 次，用于快速和W相乘
        # inputs_tiled.shape=[None, num_capsule, input_num_capsule, input_dim_capsule]
        inputs_tiled = K.tile(inputs_expand, [1, self.num_capsule, 1, 1])

        # 将inputs_tiled的batch中的每一条数据，计算inputs+W
        # x.shape = [num_capsule, input_num_capsule, input_dim_capsule]
        # W.shape = [num_capsule, input_num_capsule, dim_capsule, input_dim_capsule]
        # 将x和W的前两个维度看作'batch'维度，向量和矩阵相乘:
        # [input_dim_capsule] x [dim_capsule, input_dim_capsule]^T -> [dim_capsule].
        # inputs_hat.shape = [None, num_capsule, input_num_capsule, dim_capsutel
        inputs_hat = K.map_fn(lambda x: K.batch_dot(x, self.W, [2, 3]),elems=inputs_tiled)

        # Begin: Routing算法
        # 将系数b初始化为0.
        # b.shape = [None, self.num_capsule, self, input_num_capsule].
        b = tf.zeros(shape=[K.shape(inputs_hat)[0], self.num_capsule, self.input_num_capsule])
        
        assert self.routings > 0, 'The routings should be > 0.'
        for i in range(self.routings):
            # c.shape=[None, num_capsule, input_num_capsule]
            C = tf.nn.softmax(b ,axis=1)
            # c.shape = [None, num_capsule, input_num_capsule]
            # inputs_hat.shape = [None, num_capsule, input_num_capsule, dim_capsule]
            # 将c与inputs_hat的前两个维度看作'batch'维度，向量和矩阵相乘:
            # [input_num_capsule] x [input_num_capsule, dim_capsule] -> [dim_capsule],
            # outputs.shape= [None, num_capsule, dim_capsule]
            outputs = squash(K. batch_dot(C, inputs_hat, [2, 2])) # [None, 10, 16]
        
            if i < self.routings - 1:
                # outputs.shape = [None, num_capsule, dim_capsule]
                # inputs_hat.shape = [None, num_capsule, input_num_capsule, dim_capsule]
                # 将outputs和inρuts_hat的前两个维度看作‘batch’ 维度，向量和矩阵相乘:
                # [dim_capsule] x [imput_num_capsule, dim_capsule]^T -> [input_num_capsule]
                # b.shape = [batch_size. num_capsule, input_nom_capsule]
#                 b += K.batch_dot(outputs, inputs_hat, [2, 3]) to this b += tf.matmul(self.W, x)
                b += K.batch_dot(outputs, inputs_hat, [2, 3])

        # End: Routing 算法
        return outputs

    def compute_output_shape(self, input_shape):
        return tuple([None, self.num_capsule, self.dim_capsule])

    def get_config(self):
        config = {
            'num_capsule': self.num_capsule,
            'dim_capsule': self.dim_capsule,
            'routings': self.routings
            }
        base_config = super(DenseCapsule, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [6]:
from tensorflow import keras
from keras.regularizers import l2#正则化
x = layers.Input(shape=[512,1, 1])
#普通卷积层
conv1 = layers.Conv2D(filters=16, kernel_size=(2, 1),activation='relu',padding='valid',name='conv1')(x)

# Layer 3: 使用“squash”激活的Conv2D层， 然后重塑 [None, num_capsule, dim_vector]
primarycaps = PrimaryCap(conv1, dim_capsule=8, n_channels=12, kernel_size=(4, 1), strides=2, padding='valid')
# Layer 4: 数字胶囊层，动态路由算法在这里工作。
digitcaps = DenseCapsule(num_capsule=2, dim_capsule=16, routings=3, name='digit_caps')(primarycaps)
# Layer 5:这是一个辅助层，用它的长度代替每个胶囊。只是为了符合标签的形状。
out_caps = Length(name='out_caps')(digitcaps)

model = keras.Model(x, out_caps)    
model.summary()  

Instructions for updating:
Use fn_output_signature instead
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 1, 1)]       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 511, 1, 16)        48        
_________________________________________________________________
primarycap_conv2d (Conv2D)   (None, 254, 1, 96)        6240      
_________________________________________________________________
primarycap_reshape (Reshape) (None, 3048, 8)           0         
_________________________________________________________________
primarycap_squash (Lambda)   (None, 3048, 8)           0         
_________________________________________________________________
digit_caps (DenseCapsule)    (None, 2, 16)             780288    
_________________________________________________________________
ou

In [7]:

#定义优化
model.compile(loss='categorical_crossentropy',
              optimizer='adam',metrics=['accuracy']) 

In [ ]:
import time
time_begin = time.time()
history = model.fit(x_train,one_hot_train_labels,
                    validation_split=0.1,
                    epochs=50,batch_size=10,
                    shuffle=True)
time_end = time.time()
time = time_end - time_begin
print('time:', time)

Epoch 1/50
19/19 [==============================] - 7s 202ms/step - loss: 0.6385 - accuracy: 0.5229 - val_loss: 0.5538 - val_accuracy: 0.4286
Epoch 2/50
19/19 [==============================] - 2s 96ms/step - loss: 0.4801 - accuracy: 0.5289 - val_loss: 0.3989 - val_accuracy: 0.9524
Epoch 3/50
19/19 [==============================] - 2s 89ms/step - loss: 0.2384 - accuracy: 0.9934 - val_loss: 0.1685 - val_accuracy: 1.0000
Epoch 4/50
19/19 [==============================] - 1s 77ms/step - loss: 0.0913 - accuracy: 1.0000 - val_loss: 0.1300 - val_accuracy: 1.0000
Epoch 5/50
19/19 [==============================] - 1s 78ms/step - loss: 0.0599 - accuracy: 1.0000 - val_loss: 0.0888 - val_accuracy: 1.0000
Epoch 6/50
19/19 [==============================] - 2s 82ms/step - loss: 0.0285 - accuracy: 1.0000 - val_loss: 0.0773 - val_accuracy: 1.0000
Epoch 7/50
19/19 [==============================] - 2s 91ms/step - loss: 0.0216 - accuracy: 1.0000 - val_loss: 0.0652 - val_accuracy: 1.0000
Epoch 8/50
1

In [ ]:
import time
time_begin = time.time()
score = model.evaluate(x_test,one_hot_test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
 
time_end = time.time()
time = time_end - time_begin
print('time:', time)

In [ ]:
#绘制acc-loss曲线
import matplotlib.pyplot as plt

plt.plot(history.history['loss'],color='r')
plt.plot(history.history['val_loss'],color='g')
plt.plot(history.history['accuracy'],color='b')
plt.plot(history.history['val_accuracy'],color='k')
plt.title('model loss and acc')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train_loss', 'test_loss','train_acc', 'test_acc'], loc='center right')
# plt.legend(['train_loss','train_acc'], loc='upper left')
#plt.savefig('1.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'],color='r')
plt.plot(history.history['accuracy'],color='b')
plt.title('model loss and sccuracy ')
plt.ylabel('loss/sccuracy')
plt.xlabel('epoch')
plt.legend(['train_loss', 'train_sccuracy'], loc='center right')
plt.show()